In [2]:
!pip install pandas openpyxl

In [4]:
import pandas as pd
import os

path_filename = '/workspaces/realestatecrawler/서울시 아파트_2024-09-18.xlsx'

# 서울시 아파트_2024-09-18.xlsx 파일을 읽어옵니다.
excel_file = pd.ExcelFile(path_filename)

# 첫 번째 시트를 데이터프레임으로 변환합니다.
df = excel_file.parse(excel_file.sheet_names[0])

# 두 번째 시트를 데이터프레임으로 변환합니다.
df2 = excel_file.parse(excel_file.sheet_names[1])

# 원본 데이터의 행 수를 저장합니다.
original_rows = len(df)

# '거래종류' 열을 모두 '매매'로 변경합니다.
df['거래종류'] = '매매'

# '매매가/보증금(원)' 컬럼의 인덱스를 찾습니다.
price_column_index = df.columns.get_loc('매매가/보증금(원)')

# '매매가/보증금(원)' 컬럼 이후의 모든 컬럼에 대해 숫자 데이터를 float으로 변환합니다.
for column in df.columns[price_column_index:]:
    df[column] = pd.to_numeric(df[column].replace('(매물없음)', pd.NA), errors='coerce')

# 숫자로 변환할 수 없는 값(NaN)이 있는 행을 제거합니다.
# df = df.dropna(subset=df.columns[price_column_index:])

# 거래종류가 '매매'이고 전용면적이 84㎡ 이상인 데이터만 필터링합니다.
df_filtered = df[(df['거래종류'] == '매매') & (df['전용면적'] >= 84)]

# 필터링된 데이터를 전용면적 기준으로 내림차순 정렬합니다.
df_sorted = df_filtered.sort_values('전용면적', ascending=False)



In [5]:
# 정렬된 데이터에서 매매금액이 15억 이하인 아파트 매물리스트를 출력합니다.
result = df_sorted[df_sorted['매매가/보증금(원)'] <= 2000000000].head()
print(result)

# 필터링 및 정렬 결과 통계를 출력합니다.
print(f"\n총 데이터 수: {len(df)}")
print(f"필터링된 데이터 수 (매매 & 전용면적 84㎡ 이상): {len(df_filtered)}")
print(f"필터링된 데이터 중 15억 이하 매물 수: {len(df_sorted[df_sorted['매매가/보증금(원)'] <= 2000000000])}")

# 데이터 타입 확인
print("\n데이터 타입:")
print(df_sorted.dtypes)

       Unnamed: 0          코드   시도  시군구  읍면동    단지명 매물종류    동 거래종류  \
26546       26547  135644 332  서울시  강남구  청담동  PH129  아파트  1동    매매   
26502       26503  135644 332  서울시  강남구  청담동  PH129  아파트  1동    매매   
26505       26506  135644 332  서울시  강남구  청담동  PH129  아파트  1동    매매   
26530       26531  135644 332  서울시  강남구  청담동  PH129  아파트  1동    매매   
26515       26516  135644 332  서울시  강남구  청담동  PH129  아파트  1동    매매   

        매매가/보증금(원)  ...  계약면적(m2)  전용면적.1  전용률  방 개수  욕실 개수  해당면적세대수  현관구조  \
26546  300000000.0  ...    332.88  273.96  NaN   5.0    3.0      NaN   NaN   
26502  300000000.0  ...    332.88  273.96  NaN   5.0    3.0      NaN   NaN   
26505  300000000.0  ...    332.88  273.96  NaN   5.0    3.0      NaN   NaN   
26530  300000000.0  ...    332.88  273.96  NaN   5.0    3.0      NaN   NaN   
26515  300000000.0  ...    332.88  273.96  NaN   5.0    3.0      NaN   NaN   

        매매수   전세수  월세수  
26546  70.0  30.0  5.0  
26502  70.0  30.0  5.0  
26505  70.0  30.0  5.0  
26530  70.

In [6]:
tmp = df_sorted[df_sorted['매매가/보증금(원)'] <= 2000000000]
tmp.to_csv('/workspaces/realestatecrawler/filtered.csv', index=False, encoding='cp949')

In [7]:
df[['계약면적(m2)','전용면적']].head()

,계약면적(m2),전용면적
0,NaN,NaN
1,135.56,108.0
2,135.56,108.0
3,132.25,105.0
4,132.25,105.0


In [13]:
result.to_csv('tmp.csv', encoding='cp949', index=False)

In [8]:
[x for x in df.columns.tolist() if '전용' in x]

['전용면적', '전용면적.1', '전용률']